# NVIDIA AI Developers Day
## NVIDIA Inference Technology:<br> Deep-dive on TensorRT, Transfer Learning Toolkit and Deepstream
This notebook shows an example usecase of SSD object detection inference.<br>
[TensorRT (Session1)](#head-a) and [Transfer learning toolkit (Session2)](#head-b) will help you to optimize and quantize your network. <br>
And Deepstream will be great framework for your Real-time Intelligent Video Analysis application.


## [Sesion 3. Deepstream SDK]
This notebook shows an example usecase of deepstream.


1. [Set up env variables](#head-1) <br>
2. [Introduction to Deepstream](#head-2)<br>
3. [Getting started with deepstream](#head-3)<br>
    3.1 [End-to-end Deepstream applications](#head-3-1)<br>
    3.2 [Conguration for deepstream-app](#head-3-2)<br>
    3.3 [Deepstream applications](#head-3-3)<br>
    3.4 [YOLO for object detcetion](#head-3-4)<br>
    3.5 [SSD for object detcetion from TensorRT](#head-3-5)<br>
    3.6 [SSD for object detcetion from Transfer Learning Toolkit](#head-3-6)<br>
4. [References](#head-4)

<br />
<img src='ds_figures/DS_overview.PNG'/>
<p style="text-align: center;color:gray">Overivew of Deepstream SDK</p>

### 1. Set up env variables  <a class="anchor" id="head-1"></a>

In [1]:
import base64
import io
from IPython.display import HTML
from IPython import display as ipythondisplay

### 2. Introduction to Deepstream  <a class="anchor" id="head-2"></a>

<br />
<img src='ds_figures/DS_gstreamer_pipeline.PNG'/>
<p style="text-align: center;color:gray">Gstreamer pipeline</p>

<br />
<img src='ds_figures/DS_ds_building_block.PNG'/>
<p style="text-align: center;color:gray">Deepstream building block</p>

<br />
<img src='ds_figures/DS_ds_nvinfer.PNG'/>
<p style="text-align: center;color:gray">gst-nvinfer plugin</p>

In [2]:
%%bash
pwd
ls

/root/deepstream_sdk_v4.0.1_x86_64
LICENSE.txt
LicenseAgreement.pdf
README
deepstream.ipynb
ds_figures
out.mp4
samples
sources


### 3. Getting started with deepstream <a class="anchor" id="head-3"></a>

###  3.1 End-to-end Deepstream applications <a class="anchor" id="head-3-1"></a>

<br />
<img src='ds_figures/DS_deepstream_app.PNG'/>
<p style="text-align: center;color:gray">deepstream-app application</p>

###  3.2 Conguration for deepstream-app <a class="anchor" id="head-3-2"></a>

In [ ]:
# %load samples/configs/deepstream-app/source4_1080p_dec_infer-resnet_tracker_sgie_tiled_display_int8.txt
# Copyright (c) 2018 NVIDIA Corporation.  All rights reserved.
#
# NVIDIA Corporation and its licensors retain all intellectual property
# and proprietary rights in and to this software, related documentation
# and any modifications thereto.  Any use, reproduction, disclosure or
# distribution of this software and related documentation without an express
# license agreement from NVIDIA Corporation is strictly prohibited.

[application]
enable-perf-measurement=1
perf-measurement-interval-sec=5
#gie-kitti-output-dir=streamscl

[tiled-display]
enable=1
rows=2
columns=2
width=1280
height=720
gpu-id=0
#(0): nvbuf-mem-default - Default memory allocated, specific to particular platform
#(1): nvbuf-mem-cuda-pinned - Allocate Pinned/Host cuda memory, applicable for Tesla
#(2): nvbuf-mem-cuda-device - Allocate Device cuda memory, applicable for Tesla
#(3): nvbuf-mem-cuda-unified - Allocate Unified cuda memory, applicable for Tesla
#(4): nvbuf-mem-surface-array - Allocate Surface Array memory, applicable for Jetson
nvbuf-memory-type=0

[source0]
enable=1
#Type - 1=CameraV4L2 2=URI 3=MultiURI 4=RTSP
type=3
uri=file://../../streams/sample_1080p_h264.mp4
num-sources=4
#drop-frame-interval=2
gpu-id=0
# (0): memtype_device   - Memory type Device
# (1): memtype_pinned   - Memory type Host Pinned
# (2): memtype_unified  - Memory type Unified
cudadec-memtype=0

[sink0]
enable=0
#Type - 1=FakeSink 2=EglSink 3=File
type=2
sync=1
source-id=0
gpu-id=0
nvbuf-memory-type=0

[sink1]
enable=1
type=3
#1=mp4 2=mkv
container=1
#1=h264 2=h265
codec=1
sync=0
#iframeinterval=10
bitrate=2000000
output-file=out.mp4
source-id=0

[sink2]
enable=0
#Type - 1=FakeSink 2=EglSink 3=File 4=RTSPStreaming
type=4
#1=h264 2=h265
codec=1
sync=0
#iframeinterval=10
bitrate=400000
# set below properties in case of RTSPStreaming
rtsp-port=8554
udp-port=5400

[osd]
enable=1
gpu-id=0
border-width=1
text-size=15
text-color=1;1;1;1;
text-bg-color=0.3;0.3;0.3;1
font=Serif
show-clock=0
clock-x-offset=800
clock-y-offset=820
clock-text-size=12
clock-color=1;0;0;0
nvbuf-memory-type=0

[streammux]
gpu-id=0
##Boolean property to inform muxer that sources are live
live-source=0
batch-size=4
##time out in usec, to wait after the first buffer is available
##to push the batch even if the complete batch is not formed
batched-push-timeout=40000
## Set muxer output width and height
width=1920
height=1080
##Enable to maintain aspect ratio wrt source, and allow black borders, works
##along with width, height properties
enable-padding=0
nvbuf-memory-type=0

# config-file property is mandatory for any gie section.
# Other properties are optional and if set will override the properties set in
# the infer config file.
[primary-gie]
enable=1
gpu-id=0
model-engine-file=../../models/Primary_Detector/resnet10.caffemodel_b4_int8.engine
batch-size=4
#Required by the app for OSD, not a plugin property
bbox-border-color0=1;0;0;1
bbox-border-color1=0;1;1;1
bbox-border-color2=0;0;1;1
bbox-border-color3=0;1;0;1
interval=0
gie-unique-id=1
nvbuf-memory-type=0
config-file=config_infer_primary.txt

[tracker]
enable=1
tracker-width=640
tracker-height=368
#ll-lib-file=/opt/nvidia/deepstream/deepstream-4.0/lib/libnvds_mot_iou.so
#ll-lib-file=/opt/nvidia/deepstream/deepstream-4.0/lib/libnvds_nvdcf.so
ll-lib-file=/opt/nvidia/deepstream/deepstream-4.0/lib/libnvds_mot_klt.so
#ll-config-file required for DCF/IOU only
#ll-config-file=tracker_config.yml
#ll-config-file=iou_config.txt
gpu-id=0
#enable-batch-process applicable to DCF only
enable-batch-process=1

[secondary-gie0]
enable=1
model-engine-file=../../models/Secondary_VehicleTypes/resnet18.caffemodel_b16_int8.engine
gpu-id=0
batch-size=16
gie-unique-id=4
operate-on-gie-id=1
operate-on-class-ids=0;
config-file=config_infer_secondary_vehicletypes.txt

[secondary-gie1]
enable=1
model-engine-file=../../models/Secondary_CarColor/resnet18.caffemodel_b16_int8.engine
batch-size=16
gpu-id=0
gie-unique-id=5
operate-on-gie-id=1
operate-on-class-ids=0;
config-file=config_infer_secondary_carcolor.txt

[secondary-gie2]
enable=1
model-engine-file=../../models/Secondary_CarMake/resnet18.caffemodel_b16_int8.engine
batch-size=16
gpu-id=0
gie-unique-id=6
operate-on-gie-id=1
operate-on-class-ids=0;
config-file=config_infer_secondary_carmake.txt

[tests]
file-loop=0


In [ ]:
# %load samples/configs/deepstream-app/config_infer_primary.txt
# Copyright (c) 2018 NVIDIA Corporation.  All rights reserved.
#
# NVIDIA Corporation and its licensors retain all intellectual property
# and proprietary rights in and to this software, related documentation
# and any modifications thereto.  Any use, reproduction, disclosure or
# distribution of this software and related documentation without an express
# license agreement from NVIDIA Corporation is strictly prohibited.

# Following properties are mandatory when engine files are not specified:
#   int8-calib-file(Only in INT8)
#   Caffemodel mandatory properties: model-file, proto-file, output-blob-names
#   UFF: uff-file, input-dims, uff-input-blob-name, output-blob-names
#   ONNX: onnx-file
#
# Mandatory properties for detectors:
#   num-detected-classes
#
# Optional properties for detectors:
#   enable-dbscan(Default=false), interval(Primary mode only, Default=0)
#   custom-lib-path,
#   parse-bbox-func-name
#
# Mandatory properties for classifiers:
#   classifier-threshold, is-classifier
#
# Optional properties for classifiers:
#   classifier-async-mode(Secondary mode only, Default=false)
#
# Optional properties in secondary mode:
#   operate-on-gie-id(Default=0), operate-on-class-ids(Defaults to all classes),
#   input-object-min-width, input-object-min-height, input-object-max-width,
#   input-object-max-height
#
# Following properties are always recommended:
#   batch-size(Default=1)
#
# Other optional properties:
#   net-scale-factor(Default=1), network-mode(Default=0 i.e FP32),
#   model-color-format(Default=0 i.e. RGB) model-engine-file, labelfile-path,
#   mean-file, gie-unique-id(Default=0), offsets, gie-mode (Default=1 i.e. primary),
#   custom-lib-path, network-mode(Default=0 i.e FP32)
#
# The values in the config file are overridden by values set through GObject
# properties.

[property]
gpu-id=0
net-scale-factor=0.0039215697906911373
model-file=../../models/Primary_Detector/resnet10.caffemodel
proto-file=../../models/Primary_Detector/resnet10.prototxt
model-engine-file=../../models/Primary_Detector/resnet10.caffemodel_b30_int8.engine
labelfile-path=../../models/Primary_Detector/labels.txt
int8-calib-file=../../models/Primary_Detector/cal_trt.bin
batch-size=30
process-mode=1
model-color-format=0
## 0=FP32, 1=INT8, 2=FP16 mode
network-mode=1
num-detected-classes=4
interval=0
gie-unique-id=1
output-blob-names=conv2d_bbox;conv2d_cov/Sigmoid
#parse-bbox-func-name=NvDsInferParseCustomResnet
#custom-lib-path=/path/to/libnvdsparsebbox.so
#enable-dbscan=1

[class-attrs-all]
threshold=0.2
group-threshold=1
## Set eps=0.7 and minBoxes for enable-dbscan=1
eps=0.2
#minBoxes=3
roi-top-offset=0
roi-bottom-offset=0
detected-min-w=0
detected-min-h=0
detected-max-w=0
detected-max-h=0

## Per class configuration
#[class-attrs-2]
#threshold=0.6
#eps=0.5
#group-threshold=3
#roi-top-offset=20
#roi-bottom-offset=10
#detected-min-w=40
#detected-min-h=40
#detected-max-w=400
#detected-max-h=800


### 3.3  Deepstream applications <a class="anchor" id="head-3-3"></a>

<br />
<img src='ds_figures/DS_multiple_stream1.PNG'/>
<p style="text-align: center;color:gray">Multi-stream reference application (object decteion + tracking) </p>

In [5]:
!deepstream-app -c samples/configs/deepstream-app/source4_1080p_dec_infer-resnet_tracker_sgie_tiled_display_int8.txt


(deepstream-app:3281): GLib-GObject-WARNING **: 16:29:09.767: g_object_set_is_valid_property: object class 'nvv4l2h264enc' has no property named 'bufapi-version'
Creating LL OSD context new
gstnvtracker: Loading low-level lib at /opt/nvidia/deepstream/deepstream-4.0/lib/libnvds_mot_klt.so
gstnvtracker: Optional NvMOT_RemoveStreams not implemented
gstnvtracker: Batch processing is OFF

Runtime commands:
	h: Print this help
	q: Quit

	p: Pause
	r: Resume

NOTE: To expand a source in the 2D tiled display and view object details, left-click on the source.
      To go back to the tiled display, right-click anywhere on the window.

** INFO: <bus_callback:189>: Pipeline ready

Creating LL OSD context new
** INFO: <bus_callback:175>: Pipeline running

KLT Tracker Init
KLT Tracker Init
KLT Tracker Init
KLT Tracker Init

**PERF: FPS 0 (Avg)	FPS 1 (Avg)	FPS 2 (Avg)	FPS 3 (Avg)	
**PERF: 100.90 (100.90)	100.90 (100.90)	100.90 (100.90)	100.90 (100.90)	
**PERF: 100.86 (100.87)	100.86 (100.87)	100.86

In [ ]:
video = io.open("out.mp4", 'r+b').read()
encoded = base64.b64encode(video)
ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
            loop controls style="height: 400px;">
            <source src="data:video/mp4;base64,{0}" type="video/mp4" />
         </video>'''.format(encoded.decode('ascii'))))


<br />
<img src='ds_figures/DS_multiple_stream2.PNG'/>
<p style="text-align: center;color:gray">Multi-stream reference application (object decteion) </p>

In [7]:
!deepstream-app -c samples/configs/deepstream-app/source30_1080p_dec_infer-resnet_tiled_display_int8.txt


(deepstream-app:3428): GLib-GObject-WARNING **: 16:30:16.828: g_object_set_is_valid_property: object class 'nvv4l2h264enc' has no property named 'bufapi-version'
Creating LL OSD context new

Runtime commands:
	h: Print this help
	q: Quit

	p: Pause
	r: Resume

NOTE: To expand a source in the 2D tiled display and view object details, left-click on the source.
      To go back to the tiled display, right-click anywhere on the window.

** INFO: <bus_callback:189>: Pipeline ready

Creating LL OSD context new
** INFO: <bus_callback:175>: Pipeline running


**PERF: FPS 0 (Avg)	FPS 1 (Avg)	FPS 2 (Avg)	FPS 3 (Avg)	FPS 4 (Avg)	FPS 5 (Avg)	FPS 6 (Avg)	FPS 7 (Avg)	FPS 8 (Avg)	FPS 9 (Avg)	FPS 10 (Avg)	FPS 11 (Avg)	FPS 12 (Avg)	FPS 13 (Avg)	FPS 14 (Avg)	FPS 15 (Avg)	FPS 16 (Avg)	FPS 17 (Avg)	FPS 18 (Avg)	FPS 19 (Avg)	FPS 20 (Avg)	FPS 21 (Avg)	FPS 22 (Avg)	FPS 23 (Avg)	FPS 24 (Avg)	FPS 25 (Avg)	FPS 26 (Avg)	FPS 27 (Avg)	FPS 28 (Avg)	FPS 29 (Avg)	
**PERF: 17.41 (17.41)	18.84 (18.84)	21.91 (21.91)	25

In [ ]:
video = io.open("out.mp4", 'r+b').read()
encoded = base64.b64encode(video)
ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
            loop controls style="height: 400px;">
            <source src="data:video/mp4;base64,{0}" type="video/mp4" />
         </video>'''.format(encoded.decode('ascii'))))

### 3.4 YOLO for object detcetion <a class="anchor" id="head-3-4"></a>

<br />
<img src='ds_figures/DS_yolo.PNG'/>
<p style="text-align: center;color:gray">gst-nvinfer for darknet YOLO</p>

In [9]:
%%bash
export CUDA_VER=10.1
cd sources/objectDetector_Yolo/nvdsinfer_custom_impl_Yolo/
make clean
make

rm -rf libnvdsinfer_custom_impl_Yolo.so
g++ -o libnvdsinfer_custom_impl_Yolo.so  nvdsinfer_yolo_engine.o nvdsparsebbox_Yolo.o yoloPlugins.o trt_utils.o yolo.o kernels.o -shared -Wl,--start-group -lnvinfer_plugin -lnvinfer -lnvparsers -L/usr/local/cuda-10.1/lib64 -lcudart -lcublas -lstdc++fs -Wl,--end-group


In [10]:
%%bash
cd sources/objectDetector_Yolo
./prebuild.sh

Process is interrupted.


In [11]:
%%bash
cd  sources/objectDetector_Yolo/
deepstream-app -c deepstream_app_config_yoloV3.txt

Creating LL OSD context new
Loading pre-trained weights...
Loading complete!
Total Number of weights read : 62001757
      layer               inp_size            out_size       weightPtr
(1)   conv-bn-leaky     3 x 608 x 608      32 x 608 x 608    992   
(2)   conv-bn-leaky    32 x 608 x 608      64 x 304 x 304    19680 
(3)   conv-bn-leaky    64 x 304 x 304      32 x 304 x 304    21856 
(4)   conv-bn-leaky    32 x 304 x 304      64 x 304 x 304    40544 
(5)   skip             64 x 304 x 304      64 x 304 x 304        - 
(6)   conv-bn-leaky    64 x 304 x 304     128 x 152 x 152    114784
(7)   conv-bn-leaky   128 x 152 x 152      64 x 152 x 152    123232
(8)   conv-bn-leaky    64 x 152 x 152     128 x 152 x 152    197472
(9)   skip            128 x 152 x 152     128 x 152 x 152        - 
(10)  conv-bn-leaky   128 x 152 x 152      64 x 152 x 152    205920
(11)  conv-bn-leaky    64 x 152 x 152     128 x 152 x 152    280160
(12)  skip            128 x 152 x 152     128 x 152 x 152       


(deepstream-app:3696): GLib-GObject-WARNING **: 16:33:57.856: g_object_set_is_valid_property: object class 'nvv4l2h264enc' has no property named 'bufapi-version'
0:00:01.584569232  3696 0x55d65df6f130 INFO                 nvinfer gstnvinfer.cpp:519:gst_nvinfer_logger:<primary_gie_classifier> NvDsInferContext[UID 1]:initialize(): Trying to create engine from model files
0:01:35.148059872  3696 0x55d65df6f130 INFO                 nvinfer gstnvinfer.cpp:519:gst_nvinfer_logger:<primary_gie_classifier> NvDsInferContext[UID 1]:generateTRTModel(): Storing the serialized cuda engine to file at /root/deepstream_sdk_v4.0.1_x86_64/sources/objectDetector_Yolo/model_b1_int8.engine


In [ ]:
video = io.open("sources/objectDetector_Yolo/out.mp4", 'r+b').read()
encoded = base64.b64encode(video)
ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
            loop controls style="height: 400px;">
            <source src="data:video/mp4;base64,{0}" type="video/mp4" />
         </video>'''.format(encoded.decode('ascii'))))


### 3.5 SSD  for object detcetion from TensorRT <a class="anchor" id="head-3-5"></a>

### Prerequisites

* [Deepstream SDK 4.0+](https://developer.nvidia.com/deepstream-sdk)
 You can run deepstream-test1 sample to check Deepstream installation is successful or not.

* [TensorRT 5.1 GA](https://developer.nvidia.com/tensorrt)

* [TensorRT OSS (release/5.1 branch)](https://github.com/NVIDIA/TensorRT/tree/release/5.1)
This repository depends on the TensorRT OSS plugins. Specifically, the FasterRCNN sample depends on the `cropAndResizePlugin` and `proposalPlugin`; the MaskRCNN sample depends on the `ProposalLayer_TRT`, `PyramidROIAlign_TRT`, `DetectionLayer_TRT` and `SpecialSlice_TRT`; the SSD sample depends on the `batchTilePlugin`. To use these plugins for the samples here, complile a new `libnvinfer_plugin.so*` and replace your system `libnvinfer_plugin.so*`.


In [13]:
%%bash
export CUDA_VER=10.1
cd /root/deepstream_sdk_v4.0.1_x86_64/sources/objectDetector_SSD/nvdsinfer_custom_impl_ssd/
make clean
make

rm -rf libnvdsinfer_custom_impl_ssd.so
g++ -o libnvdsinfer_custom_impl_ssd.so nvdsparsebbox_ssd.cpp nvdsiplugin_ssd.cpp -Wall -Werror -std=c++11 -shared -fPIC -I../../includes -I/usr/local/cuda-10.1/include -Wl,--start-group -lnvinfer -lnvparsers -L/usr/local/cuda-10.1/lib64 -lcudart -lcublas -Wl,--end-group


In [14]:
%%bash
cd /root/deepstream_sdk_v4.0.1_x86_64/sources/objectDetector_SSD
deepstream-app -c deepstream_app_config_ssd_trt.txt


(deepstream-app:3732): GLib-GObject-WARNING **: 16:38:52.287: g_object_set_is_valid_property: object class 'nvv4l2h264enc' has no property named 'bufapi-version'
deepstream-app: engine.cpp:1104: bool nvinfer1::rt::Engine::deserialize(const void*, std::size_t, nvinfer1::IGpuAllocator&, nvinfer1::IPluginFactory*): Assertion `size >= bsize && "Mismatch between allocated memory size and expected size of serialized engine."' failed.
bash: line 2:  3732 Aborted                 (core dumped) deepstream-app -c deepstream_app_config_ssd_trt.txt


## Errors!!!!

In [17]:
!cp /plugins/libnvinfer/v19.09/libnvinfer* /usr/lib/x86_64-linux-gnu/

In [18]:
%%bash
export CUDA_VER=10.1
cd /root/deepstream_sdk_v4.0.1_x86_64/sources/objectDetector_SSD/nvdsinfer_custom_impl_ssd/
make clean
make

rm -rf libnvdsinfer_custom_impl_ssd.so
g++ -o libnvdsinfer_custom_impl_ssd.so nvdsparsebbox_ssd.cpp nvdsiplugin_ssd.cpp -Wall -Werror -std=c++11 -shared -fPIC -I../../includes -I/usr/local/cuda-10.1/include -Wl,--start-group -lnvinfer -lnvparsers -L/usr/local/cuda-10.1/lib64 -lcudart -lcublas -Wl,--end-group


In [ ]:
%%bash
cd /root/deepstream_sdk_v4.0.1_x86_64/sources/objectDetector_SSD
deepstream-app -c deepstream_app_config_ssd_trt.txt

In [ ]:
video = io.open("sources/objectDetector_SSD/out.mp4", 'r+b').read()
encoded = base64.b64encode(video)
ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
            loop controls style="height: 400px;">
            <source src="data:video/mp4;base64,{0}" type="video/mp4" />
         </video>'''.format(encoded.decode('ascii'))))

### 3.6 SSD  for object detcetion from Transfer Learning Toolkit <a class="anchor" id="head-3-6"></a>

In [22]:
!cp /plugins/libnvinfer/v19.05/libnvinfer* /usr/lib/x86_64-linux-gnu/

In [23]:
%%bash
export CUDA_VER=10.1
cd /root/deepstream_sdk_v4.0.1_x86_64/sources/objectDetector_SSD/nvdsinfer_custom_impl_ssd/
make clean
make

rm -rf libnvdsinfer_custom_impl_ssd.so
g++ -o libnvdsinfer_custom_impl_ssd.so nvdsparsebbox_ssd.cpp nvdsiplugin_ssd.cpp -Wall -Werror -std=c++11 -shared -fPIC -I../../includes -I/usr/local/cuda-10.1/include -Wl,--start-group -lnvinfer -lnvparsers -L/usr/local/cuda-10.1/lib64 -lcudart -lcublas -Wl,--end-group


In [ ]:
%%bash
cd /root/deepstream_sdk_v4.0.1_x86_64/sources/objectDetector_SSD
deepstream-app -c deepstream_app_config_ssd_tlt.txt

In [ ]:
video = io.open("sources/objectDetector_SSD/out.mp4", 'r+b').read()
encoded = base64.b64encode(video)
ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
            loop controls style="height: 400px;">
            <source src="data:video/mp4;base64,{0}" type="video/mp4" />
         </video>'''.format(encoded.decode('ascii'))))

### Backup!!! 

In [ ]:
!cp /plugins/libnvinfer/v19.05/libnvinfer* /usr/lib/x86_64-linux-gnu/

In [ ]:
!cp /plugins/lib_deepstream_backup/libnvinfer* /usr/lib/x86_64-linux-gnu/

In [ ]:
!cp /plugins/libnvinfer/v19.09/* /usr/lib/x86_64-linux-gnu/

### 4. References  <a class="anchor" id="head-4"></a>


- [DeepStream SDK Download](https://developer.nvidia.com/deepstream-sdk)
- [Getting Started Guide](https://docs.nvidia.com/metropolis/deepstream/4.0/dev-guide/index.html)
- [Breaking the Boundaries of Intelligent Video Analytics with DeepStream SDK 3.0](https://devblogs.nvidia.com/intelligent-video-analytics-deepstream-sdk-3-0/)
- [Multi-Camera Large-Scale Intelligent Video Analytics with DeepStream SDK](https://devblogs.nvidia.com/multi-camera-large-scale-iva-deepstream-sdk/)
- [GitHub - An example of using DeepStream SDK for redaction](https://github.com/NVIDIA-AI-IOT/redaction_with_deepstream/)
- [GitHub - DeepStream 3.0 - 360 Degree Smart Parking Application](https://github.com/NVIDIA-AI-IOT/deepstream_360_d_smart_parking_application/)

